# Cryptography (CC4017) -- Week 3

## Exercice 1

Use Python to encrypt a file in CBC mode and decrypt it. Check for success
**https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/**

In [2]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from os import urandom

key = urandom(32) # AES-256 requires a 32 byte key
iv = urandom(16) # AES block size is 16 bytes

with open('plaintext.txt', 'rb') as f:
    plaintext = f.read()

cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend)
encryptor = cipher.encryptor()

def pad(data):
    pad_len = 16 - len(data) % 16
    return data + bytes([pad_len] * pad_len)

ciphertext = encryptor.update(pad(plaintext)) + encryptor.finalize()

with open('ciphertext.txt', 'wb') as f:
    f.write(iv + ciphertext)

with open('ciphertext.txt', 'rb') as f:
    iv = f.read(16)
    ciphertext = f.read()

cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend = default_backend)
decryptor = cipher.decryptor()

def unpad(data):
    pad_len = data[-1]
    return data[:-pad_len]

plaintext_decrypted = unpad(decryptor.update(ciphertext) + decryptor.finalize())

print(key)
print(iv)
print(plaintext_decrypted)

b'\xf8\xa7\xac\x8f\xa3\x1a5\x01\xac\xd5\xfe\xaab\xf0\x94QR\x10h\xa5\xe5\xe5b{Fh\xaf0\xa8R~<'
b'\xbb\x11\x96)\x0c\xfd_\x0f\xc5B\x98\xde0\x16\xe9\xcc'
b'"CRIPTOGRAFIA E FIXE"'


## Exercice 2

Repeat this process with OpenSSL <br>
**https://www.openssl.org/docs/man1.1.1/man1/enc.html**

> key = openssl rand -hex 32 <br>
> iv = openssl rand -hex 16

# Encrypt the file
> openssl enc -aes-256-cbc -in plaintext.txt -out ciphertext.bin -K key -iv iv

# Decrypt the file
> openssl enc -aes-256-cbc -d -in ciphertext.bin -out decrypted.txt -K key -iv iv

## Exercice 3

Edit the file to change the value of (but not delete!) one byte and decrypt again.

### Exercice 3.1

What Happended?

In CBC mode, changing one byte in the ciphertext will affect the decryption of not just the block containing the altered byte but also the following block. This happens because CBC mode chains each block's encryption, using the ciphertext from the previous block. The modified byte corrupts the decryption of the current block but propagates only to the next block, meaning that the rest of the ciphertext after the next block will still decrypt correctly. The block containing the modification will completely fail decryption due to the XOR dependency on the previous block's corrupted ciphertext.

### Exercice 3.2
Could you recover a file encrypted with CBC if the IV and the first ciphertext block were corrupted or lost?

No, you cannot recover the file correctly. The initialization vector (IV) is essential for decrypting the first block, and the decryption of the first block affects subsequent blocks in CBC mode. If the IV or the first ciphertext block is lost or corrupted, the decryption process will fail for at least the first two blocks, and the data from these blocks cannot be recovered correctly.


### Exercice 3.3
Could you recover it if during a satellite transmission one bit of the ciphertext is not delivered?

If one bit is lost during transmission, CBC mode will still decrypt, but with errors. A single-bit error in transmission will cause the decryption of the corresponding block to be entirely wrong. However, the error will not propagate beyond the next block, meaning subsequent blocks can still be decrypted correctly. This is because in CBC mode, each block is dependent on the previous block's ciphertext.

### Exercice 3.4
Could you modify a byte in the middle of a CBC encrypted file without fully re-encrypting it?

No, you cannot modify a byte in the middle of a CBC-encrypted file without re-encrypting the file from that point onward. Since each block in CBC mode depends on the ciphertext of the previous block, changing any byte would require re-encrypting all subsequent blocks to maintain consistency.

## Exercice 4
Repeat the exercise with CTR mode. What are the differences?

In [3]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

key = urandom(32)
nonce = urandom(16)

# Encrypt
cipher = Cipher(algorithms.AES(key), modes.CTR(nonce), backend=default_backend)
encryptor = cipher.encryptor()
ciphertext = encryptor.update(plaintext)

# Decrypt
cipher = Cipher(algorithms.AES(key), modes.CTR(nonce), backend=default_backend)
decryptor = cipher.decryptor()
plaintext_decrypted = decryptor.update(ciphertext)

The key difference in CTR mode is that each block is encrypted independently. Thus, altering a byte in the ciphertext results in a corresponding change in the decrypted plaintext byte while leaving other parts of the plaintext intact.